# Training of a Convolutional Autoencoder #

In [1]:
import os
import datetime
import numpy as np
from utils.image_scrape import *
from utils.image_formatting import *

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D
from tensorflow.keras import layers, losses
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

We load the data, making use of our utility functions

In [2]:
artists = list(PAINTER_DICT.keys())
X, y = preprocess_images(artists=artists, n_imgs=300, normalize=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)   # We don't need labels to train the autoencoder because it is unsupervised
X_train = apply_dropout(X_train, dropout_rate=0)

print(f'X_train shape: {X_train.shape}')
print(f'X_val shape: {X_val.shape}')

pablo-picasso
william-turner
pierre-auguste-renoir
vincent-van-gogh
paul-cezanne
claude-monet
edouard-manet
jacques-louis-david
gustave-courbet
eugene-delacroix
X_train shape: (2400, 256, 256, 3)
X_val shape: (600, 256, 256, 3)


Setting up the architecture

In [3]:
input_shape = (X_train[0].shape[0], X_train[0].shape[1], X_train[0].shape[2]) # this should NOT take into account batches. Keras adds batch dimensions
activation = "relu"
regularizer = tf.keras.regularizers.L1L2(l1=0, l2=0)
n_filters = 200

def create_autoencoder():
        model = tf.keras.models.Sequential()
        model.add(Conv2D(filters=n_filters, kernel_size=(5, 5), strides=(1,1),
                         activation=activation, input_shape=input_shape,
                         padding="same", name='Conv2D_1', kernel_regularizer=regularizer))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),
                               padding='valid', name='MaxPool_1')),
        model.add(Conv2D(filters=n_filters, kernel_size=(5, 5), strides=(1,1),
                         activation=activation, padding="same", name='Conv2D_2', kernel_regularizer=regularizer))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),
                               padding='valid', name="MaxPool_2"))
        model.add(UpSampling2D(interpolation='bilinear', name='UpSample_1'))
        model.add(Conv2DTranspose(filters=n_filters, kernel_size=(5, 5),
                                  strides=(1,1), activation=activation, padding="same", name='DeConv2D_1'))
        model.add(UpSampling2D(interpolation='bilinear', name='UpSample_2'))
        model.add(Conv2DTranspose(filters=n_filters, kernel_size=(5, 5),
                                  strides=(1,1), activation=activation, padding="same", name='DeConv2D_2'))
        model.add(Conv2DTranspose(filters=3, kernel_size=(1, 1), strides=(1,1),
                                  activation=activation,
                                  name='DeConv2D_Reconstruction'))    # reconstruct image"
        return model

autoencoder = create_autoencoder()
autoencoder.compile(optimizer='adam',
                    loss=losses.MeanSquaredError(),
                    metrics=['KLDivergence'])
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_1 (Conv2D)            (None, 256, 256, 200)     15200     
_________________________________________________________________
MaxPool_1 (MaxPooling2D)     (None, 128, 128, 200)     0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 128, 128, 200)     1000200   
_________________________________________________________________
MaxPool_2 (MaxPooling2D)     (None, 64, 64, 200)       0         
_________________________________________________________________
UpSample_1 (UpSampling2D)    (None, 128, 128, 200)     0         
_________________________________________________________________
DeConv2D_1 (Conv2DTranspose) (None, 128, 128, 200)     1000200   
_________________________________________________________________
UpSample_2 (UpSampling2D)    (None, 256, 256, 200)     0

In [4]:
autoencoder.fit(X_train, X_train,
                epochs=5,
                batch_size=32,
                shuffle=True,
                validation_data=(X_val, X_val))

Epoch 1/5
75/75 [==============================] - 495s 7s/step - loss: 0.3687 - kullback_leibler_divergence: 1.3577 - val_loss: 0.0055 - val_kullback_leibler_divergence: 0.0737
Epoch 2/5
75/75 [==============================] - 482s 6s/step - loss: 0.0043 - kullback_leibler_divergence: 0.0344 - val_loss: 0.0038 - val_kullback_leibler_divergence: 0.0762
Epoch 3/5
75/75 [==============================] - 483s 6s/step - loss: 0.0040 - kullback_leibler_divergence: 0.0335 - val_loss: 0.0035 - val_kullback_leibler_divergence: 0.0636
Epoch 4/5
75/75 [==============================] - 483s 6s/step - loss: 0.0029 - kullback_leibler_divergence: 0.0284 - val_loss: 0.0026 - val_kullback_leibler_divergence: 0.0154
Epoch 5/5
75/75 [==============================] - 483s 6s/step - loss: 0.0031 - kullback_leibler_divergence: 0.0291 - val_loss: 0.0026 - val_kullback_leibler_divergence: 0.0277


Saving the model

In [5]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
autoencoder_path = f'models/autoencoder_artists_depth_{activation}_{timestamp}'
autoencoder.save(autoencoder_path)
print('Model saved')
print(timestamp)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/autoencoder_artists_depth_relu_2021-04-23-21-10/assets
Model saved
2021-04-23-21-10
